# ResBlock

A las redes les cuesta muchísimo parender funciones sencillas (como la identidad)

Entonces se les ocurrió que cada capa en lugar de calcular un valor, calcule un "error" o "residuo"

En vez de calcualr cuánto debe valer la salida, que calcule cuánto hay que sumarle a la entrada para que sea una buena salida

In [1]:
import torch.nn as nn

In [2]:
class ResBlock(nn.Module):
    def __init__( self, residual ): #residual -> f
        super().__init__()
        self.residual =  residual
        
    def forward (self, x):
        return x + self.residual(x) #Del mismo tamaño

¿Cómo lo usamos?

Hay varias maneras. Experimentar es lo mejor

In [12]:
def crear_residual (filters): #Num filtros de x
    bottleneck = (filters+1)//2
    
    residual = nn.Sequential(
        #"Comprimimos la info"
        nn.Conv2d (filters, bottleneck, kernel_size=1),
        nn.ReLU(),
        nn.BatchNorm2d(bottleneck),
        #"Conv sobre los datos comprimidos"
        nn.Conv2d (bottleneck, bottleneck, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d (bottleneck),
        #"Descomprimimos los datos"
        nn.Conv2d (bottleneck, filters, kernel_size=1),
        nn.ReLU(),
        #Es importante SIEMPRE acabar en BatchNorm
        nn.BatchNorm2d(filters)
    )
    nn.init.constant_(residual[-1].weight,0) #Incialiar el último módulo de residual como 0
    return residual

Además este último BatchNorm deberá estar inicializado con 0's (y no con 1's) para $\gamma$, como para que empiece siendo la identidad. 

In [13]:
ResBlock (crear_residual (16) )

ResBlock(
  (residual): Sequential(
    (0): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
    (7): ReLU()
    (8): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

¡Ya tenemos los ingredientes para un resnet18, resnet38!